In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## Goals for this session
### 1. Import data
### 2. Explore data
### 3. Clean Data
### 4. Extract important sub datasets from the main dataset
### 5. Decide on what models to use to get what output

# Cloud & Big Data

##


## Experiential Learning



### 1. Problem Statement
Predictive Analytics for Retail
Develop a model that predicts consumer behavior in the retail sector based on historical data. This project involves using machine learning algorithms to forecast trends and optimize inventory management.



### 2. Data Collection
This data was collected from the UCI ML Repository.


The following additional data is given along with the dataset:

**Additional Dataset Information:**
This Online Retail II data set contains all the transactions occurring for a UK-based and registered, non-store online retail between 01/12/2009 and 09/12/2011.The company mainly sells unique all-occasion gift-ware. Many customers of the company are wholesalers.

**Additional Variable Information**
InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation. 
StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product. 
Description: Product (item) name. Nominal. 
Quantity: The quantities of each product (item) per transaction. Numeric.	
InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated. 
UnitPrice: Unit price. Numeric. Product price per unit in sterling (Â£). 
CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer. 
Country: Country name. Nominal. The name of the country where a customer resides.

In [ ]:
# Import all the necessary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [ ]:
# Now let's import the data

df = pd.read_csv("/kaggle/input/online-retail-ii-uci/online_retail_II.csv", header = 0) 

# Good practice to make a copy of the original dataset so we don't tamper with the original one
retail_data = df.copy()

# Let's see what it looks like
retail_data

In [ ]:
# We see that there are 1067371 rows and 8 columns
# Those 8 columns are:
# Invoice: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.
# StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
# Description: Product (item) name. Nominal.
# Quantity: The quantities of each product (item) per transaction. Numeric.
# InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated.
# UnitPrice: Unit price. Numeric. Product price per unit in sterling (Â£).
# CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
# Country: Country name. Nominal. The name of the country where a customer resides.

In [ ]:
# Now let us look are more info about the dataset

def get_summary(dframe):
    summary_info = pd.DataFrame({
        'ColumnName': dframe.columns,
        'DataType': dframe.dtypes,
        'Values': len(dframe),
        'UniqueValues': dframe.nunique(),
        'NullValues': dframe.isnull().sum(),
        'NullPercentage': (dframe.isnull().sum() / len(dframe)) * 100
        })
    # Adjusting 'Rows' to be the same for all columns for consistency
    summary_info['Values'] = len(retail_data)
    return summary_info

get_summary(retail_data)

In [ ]:
# Here we see 53628 unique orders (Invoice), 5305 unique products, 5924 unique customers and 43 unique countries.

# We also see that there are 4382 null values in description. Which we'll have to deal with.
# We also have 243007 null values in customer ID, which we really need to handle before we do any kind of feature engineering or modelling.

# Also, the Invoice date is in object format. Let's convert that to datetime
retail_data['InvoiceDate']= pd.to_datetime(retail_data['InvoiceDate'])

get_summary(retail_data)

In [ ]:
# Let's look at more info
# How many customers, products, transactions, countries
# How many returned and cancelled orders
# Can we check if there are returned orders among the cancelled orders?
# Stock Code to see what kinds of transactions (Discount, Postage, Manual, etc)

order_stats = pd.DataFrame([{
    'Customers': len(retail_data['Customer ID'].value_counts()),
    'Products': len(retail_data['StockCode'].value_counts()),
    'Transactions': len(retail_data['Invoice'].value_counts()),
    'Countries': len(retail_data['Country'].value_counts()),
    'CancelledOrders': len(retail_data[retail_data['Invoice'].str.contains('C')].value_counts()),
    }], columns = ['Customers', 'Products', 'Transactions', 'Countries', 'CancelledOrders'], index = ['Quantity'])
order_stats

In [ ]:
# Now let's do some feature engineering. What are the additional information do we need?
# 1. Total Price

retail_fe = retail_data.copy()
retail_fe['TotalPrice'] = retail_fe['Price'] * retail_fe['Quantity']

retail_fe

In [ ]:
# A few things to clean up.
## We have many cancelled orders, which we could drop
## We also have to deal with the special codes, that correspond to other transactions like POSTAGE, etc.

## 1. Let's look at special StockCodes
list_special_codes = retail_fe[retail_fe['StockCode'].str.contains('^[a-zA-Z]+', regex=True)]['StockCode'].unique()
## 2. Let's look at cancelled orders, looking at Invoice
list_cancel_codes = retail_fe[retail_fe['Invoice'].str.startswith('C')]['Invoice'].unique()

print(len(list_special_codes))
print(len(list_cancel_codes))

In [ ]:
retail_cleaned = retail_fe[~retail_fe['StockCode'].isin(list_special_codes)].reset_index(drop = True)
retail_cleaned = retail_cleaned[~retail_cleaned['Invoice'].isin(list_cancel_codes)].sort_values(by = 'Invoice', ascending = True).reset_index(drop = True)
retail_cleaned

In [ ]:
# Now let's look at some aggregate values (works like Pivot Table)

# Let's create a DataFrame to store the data by Invoice ID
# Aggregate by 'Invoice' to calculate total basket price
invoice_summary = retail_cleaned.groupby('Invoice').agg(
    BasketPrice = ('TotalPrice', 'sum'),
    InvoiceDate=('InvoiceDate', 'first') # Assuming each invoice has a single date
).reset_index()

invoice_summary.sort_values(by = 'BasketPrice', ascending = False)

In [ ]:
# Let's set the InvoiceDate as the index to make things easier for a time series analysis
invoice_summary.set_index('InvoiceDate', inplace = True)
# Let's add some more features, like time values like Day, month, year
invoice_summary['Year'] = invoice_summary.index.year
invoice_summary['Month'] = invoice_summary.index.month
invoice_summary['Week'] = invoice_summary.index.isocalendar().week
invoice_summary['DayOfYear'] = invoice_summary.index.dayofyear
invoice_summary['Date'] = invoice_summary.index.date

invoice_summary

In [ ]:
ts_date = invoice_summary.groupby('Date').agg(
    DayOfYear = ('DayOfYear', 'first'), 
    Year = ('Year', 'first'),
    Month = ('Month', 'first'),
    Week = ('Week', 'first'),
    BasketPrice = ('BasketPrice', 'sum')
)
ts_date

In [ ]:
plt.figure(figsize=(15, 8)) 
plt.plot(ts_date.index, ts_date['BasketPrice'], label='Basket Price')

plt.title('Basket Price Whole Picture') 
plt.xlabel('Date')
plt.ylabel('Basket Price') 
plt.xticks(rotation=45)
plt.legend()
plt.show()  # Displays the plot

In [ ]:
sns.color_palette()

In [ ]:
plt.figure(figsize=(15, 8)) 

for year in [2009, 2010, 2011]:
    filtered_df = ts_date[ts_date['Year'] == year]
    plt.plot(filtered_df['DayOfYear'], filtered_df['BasketPrice'], label=str(year))


plt.title('Basket Price Each Year') 
plt.xlabel('Date')
plt.ylabel('Basket Price') 
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()  
plt.show()

In [ ]:
# For simplicity's sake let's remove the 2009 data
ts = ts_date[ts_date['Year'] != 2009]

plt.figure(figsize=(15, 8)) 


for year in [2010, 2011]:
    filtered_df = ts[ts['Year'] == year]
    plt.plot(filtered_df['DayOfYear'], filtered_df['BasketPrice'], label=str(year))

plt.title('Basket Price Each Year') 
plt.xlabel('Date')
plt.ylabel('Basket Price') 
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()  
plt.show()


In [ ]:
filtered_df = ts[((ts['Month'] > 1 ) & (ts['Month'] < 3 )) & (ts['Year'] == 2010)] 
plt.plot(filtered_df.index, filtered_df['BasketPrice'])
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Importing the model
## This is called XG Boost (eXtreme Gradient Boost) and is beneficial for complex data
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Let's do a train/test split
xgb_X_train = ts[ts['Year'] == 2010][['DayOfYear', 'Week', 'Month', 'Year']]
xgb_y_train = ts[ts['Year'] == 2010]['BasketPrice']
xgb_X_test = ts[ts['Year'] == 2011][['DayOfYear', 'Week', 'Month', 'Year']]
xgb_y_test = ts[ts['Year'] == 2011]['BasketPrice']

# Let's train the XGBoost model
model_xgb = XGBRegressor(objective='reg:squarederror')
model_xgb.fit(xgb_X_train, xgb_y_train, eval_set=[(xgb_X_test, xgb_y_test)], verbose=5)

# Predict
predictions_xgb = model_xgb.predict(xgb_X_test)

rmse_xgb = mean_squared_error(xgb_y_test, predictions_xgb, squared = False)
print(f'XGBoost RMSE: {rmse_xgb}')

In [ ]:
# Plotting for XGBoost
plt.figure(figsize=(10, 6))
plt.plot(xgb_X_test.index, xgb_y_test, label='Actual 2011')
plt.plot(xgb_X_test.index, predictions_xgb, label='XGBoost Predictions 2011', alpha=0.7)
plt.legend()
plt.title('XGBoost Predictions vs Actual 2011')
plt.show()

# RMSE for XGBoost
print(f'XGBoost RMSE: {rmse_xgb}')

In [ ]:
# Get feature importances for XGBoost
fi = pd.DataFrame(data = model_xgb.feature_importances_,
                                 index = model_xgb.feature_names_in_,
                                 columns = ['Importance']
                                )
fi.sort_values(by = 'Importance', ascending = True).plot(kind = 'barh', title = 'Feature Importances', grid = True)
print(fi)
plt.show()

In [ ]:
# Let's run another model training. This time let's add both the years, and let's have it predict the next year.

# Combine 2010 and 2011 for training
xgb_2_X_train = ts[ts['Year'].isin([2010, 2011])][['DayOfYear', 'Week', 'Month', 'Year']]
xgb_2_y_train = ts[ts['Year'].isin([2010, 2011])]['BasketPrice']

# Create a dummy 2012 feature set
# Assuming 2012 is similar to previous years, and repeating the pattern
dates_2012 = pd.date_range(start="2012-01-01", end="2012-12-31")
xgb_2_X_predict = pd.DataFrame({
    'DayOfYear': dates_2012.dayofyear,
    'Week': dates_2012.isocalendar().week,
    'Month': dates_2012.month,
    'Year': 2012
})

# Train the XGBoost model
model_2_xgb = XGBRegressor(objective='reg:squarederror')
model_2_xgb.fit(xgb_2_X_train, xgb_2_y_train)

# Predict for 2012
predictions_2012 = model_2_xgb.predict(xgb_2_X_predict)
predictions_2012

In [ ]:
# Now let us create a dataframe that is similar to existing values, and append the predicted values.
predicted_2012_df = pd.DataFrame(index=dates_2012, columns=ts_date.columns)

# Since we only have predictions for the 'BasketPrice', we fill this column with our predictions
predicted_2012_df['BasketPrice'] = predictions_2012

# All other product columns should remain NaN as we don't have predictions for individual products
# Assuming other necessary columns like 'DayOfYear', 'Week', 'Month', and 'Year' should be filled according to 2012
predicted_2012_df['DayOfYear'] = predicted_2012_df.index.dayofyear
predicted_2012_df['Week'] = predicted_2012_df.index.isocalendar().week
predicted_2012_df['Month'] = predicted_2012_df.index.month
predicted_2012_df['Year'] = 2012
predicted_2012_df['Date'] = predicted_2012_df.index.date
predicted_2012_df.set_index('Date', inplace = True)

# For columns like 'Invoice', 'CustomerID', 'Country' which are NaN because we don't have actual data
# They should stay as NaN in predicted_2012_df

# Append the predicted_2012_df to the original ts_date dataframe
ts_date_extended = pd.concat([ts_date, predicted_2012_df])

# Rename 'BasketPrice' in predicted_2012_df to 'Prediction_2012' and keep it in the new column
ts_date_extended['Prediction_2012'] = ts_date_extended['BasketPrice'].where(ts_date_extended['Year'] == 2012)
ts_date_extended

In [ ]:
# As we can see, because of lack of values, the model has given negative values at 2012-12-31. We can clip that
ts_date_extended['Prediction_2012'] = ts_date_extended['Prediction_2012'].clip(lower=0)


In [ ]:
plt.figure(figsize=(15, 8)) 

for year in [2010, 2011]:
    filtered_df = ts_date_extended[ts_date_extended['Year'] == year]
    plt.plot(filtered_df['DayOfYear'], filtered_df['BasketPrice'], label=str(year))

ts_2012 = ts_date_extended[ts_date_extended['Year'] == 2012]    
plt.plot(ts_2012['DayOfYear'], ts_2012['Prediction_2012'], label = 'Prediction for 2012')

plt.title('Basket Price Each Year with prediction') 
plt.xlabel('Date')
plt.ylabel('Basket Price') 
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()  
plt.show()

In [ ]:
# As we can see that our model has predicted pretty accurate values for 2012, based on 2010 and 2012 data.
## But we don't can't validate its accuracy because we don't have actual values.

In [ ]:
# Let's look at the feature importances
# Get feature importances for XGBoost
fi_2 = pd.DataFrame(data = model_2_xgb.feature_importances_,
                                 index = model_xgb.feature_names_in_,
                                 columns = ['Importance']
                                )
fi_2.sort_values(by = 'Importance', ascending = True).plot(kind = 'barh', title = 'Feature Importances', grid = True)
print(fi)
plt.show()

In [ ]:
# It gives similar importances as the previous model. 

In [ ]:
# Now let us do something more
## Let us try to predict which products are bought from which customers from which countries.

# For that let's only consider the top 30 products, top 30 customers and top 10 countries and see who buys how much and when.

#So we'll have to prune the data to make it more manageable

# First, we will identify the top 30 selling products based on the total quantity sold.

# Group by StockCode and sum quantities, then get the top 50.
top_products = retail_cleaned.groupby('StockCode').agg({'Quantity': 'sum'}).nlargest(50, 'Quantity')

# Now, identify the top 10 countries by sales volume.
top_countries = retail_cleaned.groupby('Country').agg({'TotalPrice': 'sum'}).nlargest(10, 'TotalPrice')

# Lastly, identify the top 30 customers by sales volume.
top_customers = retail_cleaned.groupby('Customer ID').agg({'TotalPrice': 'sum'}).nlargest(30, 'TotalPrice')

# Filter the original cleaned dataset to include only the entries corresponding to the top categories identified.
filtered_data = retail_cleaned[
    (retail_cleaned['StockCode'].isin(top_products.index)) &
    (retail_cleaned['Country'].isin(top_countries.index)) &
    (retail_cleaned['Customer ID'].isin(top_customers.index))
].reset_index(drop = True)
filtered_data

In [ ]:
get_summary(filtered_data)

In [ ]:
# We will pivot the filtered data to get the required structure:
# - One column for each of the top 30 products indicating the quantity purchased in that invoice.
# - One binary column for each of the top 10 countries indicating if the order was placed from that country.
# - One binary column for each of the top 30 customers indicating if the order was made by that customer.
# - Columns for 'BasketPrice' and 'InvoiceDate'.

# Preparing product quantity columns
product_pivot = filtered_data.pivot_table(index='Invoice', columns='StockCode', values='Quantity', aggfunc='sum', fill_value=0)

# Preparing binary columns for countries
country_pivot = pd.get_dummies(filtered_data['Country']).groupby(filtered_data['Invoice']).max()

# Preparing binary columns for customers
customer_pivot = pd.get_dummies(filtered_data['Customer ID'], prefix='Customer').groupby(filtered_data['Invoice']).max()

# Calculating BasketPrice for each invoice
basket_price = filtered_data.groupby('Invoice').agg({'TotalPrice': 'sum'}).rename(columns={'TotalPrice': 'BasketPrice'})

# Adding InvoiceDate
invoice_date = filtered_data.groupby('Invoice').agg({'InvoiceDate': 'first'})

# Concatenating all the pivots and the basket price to form the final aggregated dataframe
aggregated_data = pd.concat([product_pivot, country_pivot, customer_pivot, basket_price, invoice_date], axis=1)

# Checking the structure of the resulting dataframe
aggregated_data.head(), aggregated_data.shape